**Lab Exercise: Analyzing seismic data in record sections [record section]** <br/>
GEOS 626/426: Applied Seismology, Carl Tape

**Instructions**

- The goal is to create different seismic record section plots. A seismic record section is a series of
seismograms that is plotted in some particular order. Typically, the x-axis is time, and then the seismograms are
separated in the y-direction and ordered by source-station distance or source-station azimuth.
- There are two main procedures of this lab:
    - fetching waveforms from the IRIS Data Management Center.
    - plotting record sections of seismograms.
- The two main procedures are executed using the software package [PySEP](https://adjtomo.github.io/pysep), which uses
[ObsPy](https://docs.obspy.org/). Check PySEP's
[data gathering](https://pysep.readthedocs.io/en/devel/autoapi/pysep/pysep/index.html#pysep.pysep.Pysep) and
[record section plot](https://pysep.readthedocs.io/en/devel/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection)
pages for details on the two procedures.
- The bandpass filter that is applied to the seismograms can have a dramatic effect on what is visible. The frequency
limits of the bandpass is one of several important choices that are needed when plotting record sections. Check the
IRIS [webpage](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming) for the SEED format seismic channel
naming.
- We will not be removing instrument responses, as we did in lab\_response. However, removing the instrument response
can easily be achieved with the same tools. The two reasons we do not remove the responses are:
    - it takes a bit longer, computationally.
    - it can significantly distort the shape of the unfiltered waveforms, especially ones that are “odd,” which, here,
      we are interested in.

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import os
import warnings

from obspy import read
from obspy.core import UTCDateTime
from pysep import Pysep
from pysep.recsec import plotw_rs

In [ ]:
# script settings

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = 12, 8

In the subsequent code cell, set *example\_index* corresponding to the example from the following list, that you
want to run -

    1   - Yahtse glacier calving event <br/>
    2   - Mw 7.5 earthquake in southeastern Alaska <br/>
    3   - explosion in Fairbanks <br/>
    4   - very low frequency (VLF) event near Kantishna <br/>
    5   - landslide near Lituya Bay <br/>
    6   - Mw 8.6 Indian Ocean (offshore Sumatra) recorded in Alaska, plus triggered earthquakes <br/>
    6.1 - triggered earthquake (Andreanof) <br/>
    6.2 - triggered earthquake (Nenana crustal) <br/>
    6.3 - triggered earthquake (Iliamna intraslab) <br/>
    7   - your own example

Now run all cells below.

>__NOTE:__ Run the whole notebook everytime you change the *example_index* or if you make other changes to the script.
> This will ensure that all variables are set correctly before each run.

>__NOTE:__ The notebook is designed such that data for a given example is downloaded only once and stored for
> subsequent use.

**Exercises**

Spend some time to understand the two key parts of each example - <br/>
a) waveform extraction, <br/>
b) plotting a record section <br/>

- What does NET.STA.LOC.CHA|AZ|DIST represent?
- What do HHZ and BHZ channels represent? Check out
[here](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming) for details.
- What input variables are needed to specify the bandpass? <br/>
- How is a bandpass filter applied within plotw\_rs()? Hint: find the online documentation.
- In the record section generated in the next section, how are the seismograms aligned?
- Try out the different options for sorting seismograms in record sections, following the documentation for
[PySEP](https://adjtomo.github.io/pysep/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection). You can add
\_r to reverse the sorting order; for example, for alphabetical\_r the sorting will go from Z to A.

In [ ]:
# choose example to run
example_index = 2

In [ ]:
# list of networks to retrieve data from
# providing an explicit list of networks is safer than using the wildcard (*)
networks      = 'AK,AT,AU,AV,BK,CI,CN,CU,GT,IC,II,IM,IU,MS,TA,TS,US,XE,XM,XR,YM,YV,XF,XP,XZ'
# networks      = '*'

In [ ]:
# setting pysep's data download defaults

download_defaults   = dict( networks                     = networks,
                            stations                     = '*',
                            locations                    = '*',
                            channels                     = 'BHZ',
                            maxdistance_km               = 200,
                            remove_clipped               = False,
                            remove_insufficient_length   = False,
                            fill_data_gaps               = 0,
                            remove_response              = False,
                            log_level                    = 'INFO',
                            write_files                  = 'sac',
                            plot_files                   = 'map',
                            output_dir                   = 'datawf',
                            sac_subdir                   = '',
                            overwrite_event_tag          = f'Example_{example_index}',
                            overwrite                    = False )

In [ ]:
# setting pysep's record section plotting defaults

plotting_defaults   = dict( pysep_path                   = f'datawf/Example_{example_index}',
                            sort_by                      = 'distance',
                            scale_by                     = 'normalize',
                            amplitude_scale_factor       = 1,
                            time_shift_s                 = None,
                            preprocess                   = 'st',
                            max_traces_per_rs            = None,
                            distance_units               = 'km',
                            tmarks                       = [0],
                            save                         = '',
                            log_level                    = 'CRITICAL' )

**Example 1: Yahtse glacier calving event**

    - From the station map, examine the source–station geometry, especially closest to the epicenter.
    - Describe the characteristics of this signal. Do you see a distinct P wave on any seismogram? (This will be
      clearer later, after you have seen P waves from normal earthquakes.)
    - What does the spectrogram show? Try changing the station (e.g., to BOOM) and other input parameters. How close is
      BOOM to the event location?
    - Describe some oddities within the record section.

In [ ]:
# 1. Yahtse Glacier event
# event information could not be found on catalog

if example_index == 1:
    
    event      = dict( origin_time                  = UTCDateTime("2010,9,18,14,15,2"),
                       event_latitude               = 60.155496,
                       event_longitude              = -141.378343,
                       event_depth_km               = 0.1,
                       event_magnitude              = 0.1 )

    duration   = dict( seconds_before_ref           = 20,
                       seconds_after_ref            = 70 )
    
    download_defaults['channels']                   = 'HHZ,BHZ'
    
    bandpass   = dict( min_period_s                 = 0.1,
                       max_period_s                 = 2 )

**Example 2: Mw 7.5 earthquake in southeastern Alaska, near-source recordings**

    - Comment on the notable features of these seismograms.

In [ ]:
# 2. Mw 7.5 earthquake in southeastern Alaska
# event information - https://earthquake.usgs.gov/earthquakes/eventpage/ak0138esnzr

if example_index == 2:
    
    channels_1 = 'BHZ,BHE,BHN,BH1,BH2'                                                         # broadband channels
    channels_2 = 'BNZ,BNE,BNN,BN1,BN2,BLZ,BLE,BLN,BL1,BL2'                                     # strong motion channels
    channels_3 = 'HNZ,HNE,HNN,HN1,HN2,HLZ,HLE,HLN,HL1,HL2'                                     # strong motion channels
    channels   = f'{channels_1},{channels_2},{channels_3}'                                     # warning: waveforms will have different units (nm/s, nm/s^2)
    
    event      = dict( origin_time                  = UTCDateTime("2013,1,5,8,58,32"),
                       event_latitude               = 55.228,
                       event_longitude              = -134.859,
                       event_depth_km               = 8.7,
                       event_magnitude              = 7.5 )
    
    duration   = dict( seconds_before_ref           = 50,
                       seconds_after_ref            = 300 )
    
    download_defaults['channels']                   = channels
    download_defaults['maxdistance_km']             = 500
    
    bandpass   = dict( min_period_s                 = None,
                       max_period_s                 = None )
    
    plotting_defaults['amplitude_scale_factor']     = 0.5
    plotting_defaults['max_traces_per_rs']          = 13

**Example 3: Explosion in Fairbanks**

    - There are two signals that appear at most stations. Start by examining the station MDM (Murphy Dome).
    - There is only one source, so how can you explain both signals in terms of their travel times and amplitudes?

In [ ]:
# 3. explosion in Fairbanks
# event information could not be found on catalog

if example_index == 3:
    
    #event location based on infrasound 
    #elat = 64.8156; elon = -147.9419                                                          # original AEC
    #elat = 64.8045; elon = -147.9653                                                          # reviewed AEC
    
    event      = dict( origin_time                  = UTCDateTime("2013,2,3,1,10,31"),
                       event_latitude               = 64.80175,
                       event_longitude              = -147.98236,
                       event_depth_km               = 0.1,
                       event_magnitude              = 0.1 )
       
    duration   = dict( seconds_before_ref           = 50,
                       seconds_after_ref            = 200 / 0.3 )                              # air wave travel time
    
    download_defaults['channels']                   = 'SHZ,HHZ,BHZ'                            # broadband channels
    
    bandpass   = dict( min_period_s                 = 0.2,
                       max_period_s                 = 1 )

**Example 4: Very low frequency earthquake near Denali**

    - Estimate the dominant frequency of this event? (Hint: use the spectrogram tool.)

In [ ]:
# 4. very low frequency (VLF) event near Kantishna
# event information taken from IRIS

if example_index == 4:
    
    event      = dict( origin_time                  = UTCDateTime("2014,1,22,12,14,34"),
                       event_latitude               = 63.46,
                       event_longitude              = -150.11,
                       event_depth_km               = 38.1,
                       event_magnitude              = 1.6 )
    
    duration   = dict( seconds_before_ref           = 0,
                       seconds_after_ref            = 100 )
    
    bandpass   = dict( min_period_s                 = None,
                       max_period_s                 = 2 )

**Example 5: Landslide near Lituya**

    - What is the dominant frequency of this event?

In [ ]:
# 5. landslide near Lituya Bay
# event information taken from IRIS

if example_index == 5:
    
    event      = dict( origin_time                  = UTCDateTime("2014,2,16,14,24,30"),
                       event_latitude               = 58.67,
                       event_longitude              = -136.84,
                       event_depth_km               = 0.1,
                       event_magnitude              = 2.4 )

    duration   = dict( seconds_before_ref           = 0,
                       seconds_after_ref            = 600 )
    
    download_defaults['maxdistance_km']             = 1000
    
    bandpass   = dict( min_period_s                 = 10,
                       max_period_s                 = 40 )

**Example 6: Mw 8.6 Indian Ocean (offshorer Sumatra) earthquake triggering earthquakes in Alaska**

Event information - <br/> 6. Indian Ocean -
https://earthquake.usgs.gov/earthquakes/eventpage/official20120411083836720_20 <br/>

    - Examine the record section and try to determine what you see.
    - For each event (which we define as a signal that appears on several stations), determine what the closest
      station is. Where did each event occur?
    - Change the bandpass period range (min_period_s and max_period_s) for the record section plot to be 2–1000 s, so
      that you see the complete frequency range of this waveform.
    - Approximately how long did this earthquake last in Alaska?

**Example 6.1, 6.2 & 6.3: Triggered earthquakes during the Mw 8.6 Indian Ocean (offshorer Sumatra) earthquake in
Alaska**

Event information - <br/>
6.1. Andreanof - https://earthquake.usgs.gov/earthquakes/eventpage/usp000jhh4 <br/>
6.2. Nenana - https://earthquake.usgs.gov/earthquakes/eventpage/ak0124ouaxa8 <br/>
6.3. Iliamna - https://earthquake.usgs.gov/earthquakes/eventpage/ak0124ouezxl <br/>

    - You are given the source parameters for three earthquakes that occurred in Alaska during the ground motion of the
      main wavetrain from the Mw 8.6 Indian Ocean (offshore Sumatra) earthquake. For each possibly triggered event,
      tabulate the following information:
        - the closest station (and the distance in km)
        - the suspicious stations
        - the widest period range over which the signal is clearly visible. This can be achieved by varying
          min_period_s and max_period_s provided as an input for plotting the record sections. You do not need to
          re-extract the waveforms.

In [ ]:
if example_index in [6, 6.1, 6.2, 6.3]:

    # origin times of known earthquakes
    origin_time_sumatra   = UTCDateTime("2012,4,11,8,38,36")
    origin_time_andreanof = UTCDateTime("2012,4,11,9,0,9")
    origin_time_nenana    = UTCDateTime("2012,4,11,9,21,57")
    origin_time_iliamna   = UTCDateTime("2012,4,11,9,40,58")
    

In [ ]:
if example_index == 6:

    # origin times, in seconds, relative to Sumatra origin time
    t_andreanof = origin_time_andreanof - origin_time_sumatra
    t_nenana    = origin_time_nenana    - origin_time_sumatra
    t_iliamna   = origin_time_iliamna   - origin_time_sumatra

    # Indian Ocean (offshore Sumatra) Mw 8.6
    event      = dict( origin_time                  = origin_time_sumatra,
                       event_latitude               = 2.327,
                       event_longitude              = 93.063,
                       event_depth_km               = 20,
                       event_magnitude              = 8.6 )

    duration   = dict( seconds_before_ref           = 0.25 * 60 * 60,
                       seconds_after_ref            = 2    * 60 * 60 )

    stations   = dict( minlatitude                  =    64.922 - 25,
                       maxlatitude                  =    64.922 + 25,
                       minlongitude                 = - 148.946 - 25,
                       maxlongitude                 = - 148.946 + 25 )
    
    download_defaults['maxdistance_km']             = 6371 * np.pi
    download_defaults                               = {**download_defaults, **stations}
    
    # P wave + triggered events
    bandpass   = dict( min_period_s                 = 0.25,
                       max_period_s                 = 0.5 )

    # full wavetrain (no triggered events visible)
    # bandpass   = dict( min_period_s                 = 2,
    #                    max_period_s                 = 1000 )

    plotting_defaults['distance_units']             = 'deg'
    plotting_defaults['tmarks']                     = [0, t_andreanof, t_nenana, t_iliamna]


In [ ]:
if example_index == 6.1:

    # triggered earthquake - Andreanof (NEIC)    
    event      = dict( origin_time                  = origin_time_andreanof,
                       event_latitude               = 51.364,
                       event_longitude              = -176.097,
                       event_depth_km               = 20.8,
                       event_magnitude              = 5.5 )

    duration   = dict( seconds_before_ref           = 10,
                       seconds_after_ref            = 600 )
    
    download_defaults['maxdistance_km']             = 2000

    bandpass   = dict( min_period_s                 = 0.25,
                       max_period_s                 = 0.5 )
    

In [ ]:
if example_index == 6.2:

    # triggered earthquake - Nenana crustal (NEIC)
    event      = dict( origin_time                  = origin_time_nenana,
                       event_latitude               = 64.922,
                       event_longitude              = -148.946,
                       event_depth_km               = 19.3,
                       event_magnitude              = 3.9 )

    duration   = dict( seconds_before_ref           = 10,
                       seconds_after_ref            = 200 )

    bandpass   = dict( min_period_s                 = 0.25,
                       max_period_s                 = 0.5 )
    

In [ ]:
if example_index == 6.3:

    # triggered earthquake - Iliamna intraslab (NEIC)
    event      = dict( origin_time                  = origin_time_iliamna,
                       event_latitude               = 60.104,
                       event_longitude              = -152.832,
                       event_depth_km               = 101.5,
                       event_magnitude              = 2.9 )

    duration   = dict( seconds_before_ref           = 10,
                       seconds_after_ref            = 200 )

    download_defaults['maxdistance_km']             = 400
    
    bandpass   = dict( min_period_s                 = 0.25,
                       max_period_s                 = 0.5 )
        

**Example 7: Your own example**

    - Take a look at any event of your interest by extracting waveforms and plotting a record section.

In [ ]:
if example_index == 7: 
    
    # your own example below
    
    event      = dict()
    
    duration   = dict()
    
    bandpass   = dict()

In [ ]:
# download data

data_dir  = f'{download_defaults["output_dir"]}/{download_defaults["overwrite_event_tag"]}'
overwrite = f'{download_defaults["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**duration,**download_defaults)
    ps.run()
else:
    print('\ndata directory already exists, proceeding to the next code cell\n')

In [ ]:
# plot source station map

plt.figure()
source_station_map = img.imread(f'datawf/Example_{example_index}/station_map.png')
plt.imshow(source_station_map);

In [ ]:
# plot the record section using Pyseps's record section plotting tool

plotw_rs(**plotting_defaults, **bandpass)

In [ ]:
# seismogram sorting options
# set to run only when for example_index = 1

if example_index == 1:
    
    sort_by_tag = ['distance', 'absolute distance', 'azimuth', 'absolute azimuth']
    
    for i, sort_by in enumerate(['distance', 'abs_distance', 'azimuth', 'abs_azimuth']):
        
        print(f'\n\nCase {i+1}: Seismograms sorted by {sort_by_tag[i]}\n\n')
        
        plotting_defaults['sort_by']   = sort_by
        
        plotw_rs(**plotting_defaults, **bandpass)

In [ ]:
# seismograms alignment on the S wave
# set to run only when for example_index = 1
    
if example_index == 1:    

    print(f'\n\nSeismograms aligned on the S wave arrival\n\n')
    
    plotting_defaults['sort_by']      = 'distance'
    plotting_defaults['time_shift_s'] = 's_arrival_time'
    
    plotw_rs(**plotting_defaults, **bandpass)

In [ ]:
# plottiing the spectrogram for a selected seismogram
# Check https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.specgram.html for details on spectrogram plotter

example_index = example_index                                                                  # select example

network  = '*'                                                                                 # select network
station  = '*'                                                                                 # select station
location = '*'                                                                                 # select location
channel  = '*'                                                                                 # select channel

sac_file = f'./{data_dir}/Example_{example_index}.{network}.{station}.{location}.{channel}.sac'
st       = read(sac_file, 'SAC')

t                  = st[0].times()
data               = st[0].data
sampling_frequency = st[0].stats.sampling_rate

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8,6))     
ax1.plot(t, data, 'k', linewidth=0.5)

image = ax2.specgram(st[0], Fs=sampling_frequency, noverlap=int(0.8*256), cmap="jet")
ax2.set_xlabel('Time - Seconds')
ax2.set_ylabel('Frequency (Hz)')

ax3 = fig.add_axes([0.9, 0.1, 0.03, 0.37])
plt.colorbar(mappable=image[3], cax=ax3)
plt.ylabel('Relative Amplitude (dB)')
plt.show()

title    = f'{st[0].stats.network}.{st[0].stats.station}.{st[0].stats.location}.{st[0].stats.channel} − starting {st[0].stats["starttime"]}'
fig.suptitle(title)

# note: the time axis of the spectrogram may not correspond to the time axis of the record sections
# this is because the spectrogram plotter assigns a default value of time = 0 to the first sample of the input data

if example_index == 1:
    fig.canvas.draw()
    labels = np.arange(-40,100,20)
    ax2.set_xticklabels(labels)